In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import math

## Linear Isotherm

In [ ]:
def lin_isotherm_system(K, J, I, tauT, tauS, beta, aS, D):
    
    # mesh sizes
    dx = 1 / K
    dz = 1 / J
    dr = 1 / I

    # ODE counts
    CTn = K + 1                     
    CSn = (K + 1) * J             
    yn  = (K + 1) * (J + 1) * I     
    N = CTn + CSn + yn

    # index mappings
    def CTindex(k):
        return k

    def CSindex(k, j):
        return CTn + k * J + (j - 1)

    def ypindex(k, j, i):
        return CTn + CSn + (k * (J + 1) + j) * I + i

    # change CS=CT at z=0 boundary
    def getCS(x, k, j):
        if j == 0:
            return x[CTindex(k)]
        else:
            return x[CSindex(k, j)]

    def system(t, x):
        rhs = np.zeros_like(x)

        # Region I: CT

        # bc at x=0
        rhs[CTindex(0)] = 0

        # equation
        for k in range(1, K + 1):
            rhs[CTindex(k)] = -1 / tauT * (((x[CTindex(k)] - x[CTindex(k-1)]) / dx) -(beta)* ((x[CSindex(k,1)] - x[CTindex(k)]) / dz))

        # Region II: CS
        for k in range(0, K + 1):
            for j in range(1, J + 1):
                middle = x[CSindex(k,j)]

                # bc at z=0
                if j == 1:
                    below = x[CTindex(k)]
                else:
                    below = x[CSindex(k, j - 1)]

                # bc at z=1
                if j == J:
                    ddCS = 2 * (below - middle) / dz ** 2
                else:
                    above = x[CSindex(k, j + 1)]
                    ddCS = (above - 2 * middle + below) / dz ** 2

                # equation
                rhs[CSindex(k, j)] = 1/tauS*(ddCS - aS ** 2 * ((x[CSindex(k,j)] - x[ypindex(k,j,I-1)]) / dr - x[CSindex(k,j)]))

        # Region III: yp=Cp*r
        for k in range(0, K + 1):
            for j in range(0, J + 1):

                # get CS value used for r=1 boundary 
                CS = getCS(x, k, j)

                for i in range(0, I):
                    middle = x[ypindex(k, j, i)]

                    # bc y=0
                    if i == 0:
                        rhs[ypindex(k, j, i)]=0
                    else:
                        left = x[ypindex(k, j, i - 1)]

                        #bc y=1
                        if i == I - 1:
                            right = CS
                        else:
                            right = x[ypindex(k, j, i + 1)]

                        #equation
                        rhs[ypindex(k, j, i)] = D * (right - 2*middle + left) / dr**2

        return rhs
    extra = [N, getCS]


    return system, extra


## D<<1

In [ ]:
def small_diffusion_system(K, J, tauT, tauS, beta, aS, D):
    
    # mesh sizes
    dx=1/K
    dz=1/J

    # ODE counts
    n_CT = K + 1     
    n_CS = (K + 1) * J     

    N = n_CT + n_CS

    # index mappings
    def idx_CT(k):
        return k

    def idx_CS(k, j):
        return n_CT + k * J + (j-1)

    # change CS=CT at z=0 boundary
    def getCS(x, k, j):
        if j == 0:
            return x[idx_CT(k)]
        else:
            return x[idx_CS(k, j)]

    def system(t, x):
        rhs = np.zeros_like(x)

        #Region I: CT

        #bc at x=0
        rhs[idx_CT(0)] = 0

        # equation
        for k in range(1, K + 1):
            rhs[idx_CT(k)] = -1 / tauT * (((x[idx_CT(k)] - x[idx_CT(k-1)]) / dx) -(beta) * ((x[idx_CS(k, 1)] - x[idx_CT(k)]) / dz))

        #Region II: CS
        for k in range(0, K + 1):
            for j in range(1, J + 1):
                middle = x[idx_cs]

                # bc at z=0
                if j == 1:
                    below = x[idx_CT(k)]
                else:
                    below = x[idx_CS(k, j - 1)]

                # bc at z=1
                if j == J:
                    ddCS = 2 * (below - middle) / dz ** 2
                else:
                    above = x[idx_CS(k, j + 1)]
                    ddCS = (above - 2*middle +below) / dz ** 2

                rhs[idx_cs] = 1/tauS*(ddCS - aS ** 2 * ((middle) / np.sqrt(np.pi * D * t)))
 
        return rhs
    extra = [N,getCS]
    return system, extra

Early time behaviour (initial condition)

In [ ]:
gam = beta / (tauT * np.sqrt(np.pi * tauS))

In [ ]:
def initial_condition(K, J, gam, t0, tauT, tauS):
    dx = 1 / K
    dz = 1 / J

    n_CT = K + 1
    n_CS = (K + 1) * J
    N = n_CT + n_CS

    y0 = np.zeros(N)
    y0[0]=1

    #CTstar
    for k in range(1,n_CT):
        xk = k * dx
        if t0 > tauT*xk: 
            y0[k] = np.exp(-2*gam*np.sqrt(t0))

    #CSstar
    for k in range(K + 1):
        xk = k * dx
        if t0 > tauT*xk:
            for j in range(1, J + 1):
                zj = j * dz
                idx = n_CT + k * J + (j - 1)
                y0[idx] = np.exp(-2 * gam * np.sqrt(t0)) * (1 - math.erf(zj / (2 * np.sqrt(tauS * t0))))

    return y0

## Full System

In [ ]:
def full_system(K, J, I, tauT, tauS, beta, aS, aP, kappa, chi):

    # mesh sizes
    dx=1/K
    dz=1/J
    dr=1/I

    # ODE counts
    CTn = K + 1             
    CSn = (K + 1) * J            
    CPn  = (K + 1) * (J + 1) * I  
    GammaPn = (K + 1) * (J + 1) * (I + 1)

    N = CTn + CSn + CPn + GammaPn

    # index mappings
    def CTindex(k):
        return k

    def CSindex(k, j):
        return CTn + k * J + (j - 1)

    def CPindex(k, j, i):
        return CTn + CSn + (k * (J + 1) + j) * I + i
    
    def gampindex(k,j,i):
        return CTn + CSn + CPn + (k * (J + 1) + j) * (I + 1) + i 

    # change CS=CT at boundary
    def getCS(x, k, j):
        if j == 0:
            return x[CTindex(k)]
        else:
            return x[CSindex(k, j)]
    
    # change CP=CS at boundary
    def getCP(x, k, j, i):
        if j == 0:
            return x[CTindex(k)]
        elif i == I:
            return x[CSindex(k, j)]
        else:
            return x[CPindex(k, j, i)]
        
    def system(t, x):
        rhs = np.zeros_like(x)

        # Region I: CT

        # bc at x=0
        rhs[CTindex(0)] = 0 

        # equation
        for k in range(1, K + 1):
            rhs[CTindex(k)] = -1.0 / tauT * (((x[CTindex(k)] - x[CTindex(k-1)]) / dx) -(beta)* ((x[CSindex(k,1)] - x[CTindex(k)]) / dz))

        # Region II:CS

        for k in range(0, K + 1):
            for j in range(1, J + 1):
                middle = x[CSindex(k,j)]

                # bc at z=0
                if j == 1:
                    below = x[CTindex(k)] 
                else:
                    below = x[CSindex(k, j - 1)]

                # bc at z=1
                if j == J:
                    
                    CSpp = 2* (below - middle) / dz ** 2
                else:
                    above = x[CSindex(k, j + 1)]
                    CSpp = (above - 2* middle + below) / dz ** 2
                
                # equation
                rhs[CSindex(k, j)] = 1/tauS*(CSpp - aS ** 2 * ((x[CSindex(k,j)] - x[CPindex(k,j,I-1)]) / dr))

        # Region III: CP
        for k in range(0, K + 1):
            for j in range(0, J + 1):
                
                # get CS value used for r=1 boundary 
                CS = getCS(x, k, j)

                # bc at r=0
                rhs[CPindex(k,j,0)] = 6 * (x[CPindex(k,j,1)] - x[CPindex(k,j,0)]) / dr**2 - aP**2 * (kappa * x[CPindex(k,j,0)] * (1 - x[gampindex(k,j,0)]) - x[gampindex(k,j,0)])
                for i in range(1, I):
                    middle = x[CPindex(k, j, i)]
                    left = x[CPindex(k, j, i - 1)]

                    # bc at r=1
                    if i == I - 1:
                        right = CS
                    else:
                        right = x[CPindex(k, j, i + 1)]

                    rhs[CPindex(k, j, i)] = ((i+1)*right - 2*i*middle + (i-1)*left) / (i*dr ** 2) -aP**2*(kappa * x[CPindex(k,j,i)] * (1 - x[gampindex(k,j,i)]) - x[gampindex(k,j,i)])


        # Adsorbed chemical: Gamma_P
        for k in range(0, K + 1):
            for j in range(0, J + 1):
                for i in range(0, I + 1):
                    #get value of CP
                    CP=getCP(x,k,j,i)
                    rhs[gampindex(k, j, i)]=chi*aP**2*(kappa*CP*(1-x[gampindex(k,j,i)])-x[gampindex(k,j,i)])
        
                    
        return rhs
    extra = [N, getCS, getCP]

    return system, extra